In [1]:
## imports and fixed parameters

import ipywidgets as widgets
from IPython.display import display, IFrame
import requests

BASE_URL = 'https://cmcl.dome40.io/'  # DOME platform URL
API_KEY = "123456"  # hard-coded for now
IFRAME_URL = 'http://192.168.1.171:4242/visualisation'


Textarea(value='', description='Text:', placeholder='Type something')

Button(button_style='success', description='Submit', icon='check', style=ButtonStyle(), tooltip='Submit')

In [ ]:
## DOME-related functions

def get_data_from_dome(provider_id, search_string):

    res_conn_query = requests.get(
        f"{BASE_URL}api/discover/results/{provider_id}",
        headers={'apikey': API_KEY},
        params={'search_string': search_string},
        timeout=10,
    )

    try:
        return res_conn_query.json()
    except:
        return res_conn_query.content


def get_emission_data(search_string):

    search_string = search_string or "ship"

    return get_data_from_dome("CMCL", search_string)


def parse_emission_data(json_emission):
    dict_emission = {}
    metadata = json_emission['metadata']
    data = json_emission['data']
    for i in range(len(metadata)):
        list_output = (split_string(
            metadata[i]['Title'], ['emission in', 'at']))
        scope = data[i]['Scope']
        label = f"{list_output[1]}: {scope}"
        timestamp = list_output[2]
        if label not in dict_emission:
            dict_emission[label] = [timestamp]
        else:
            if timestamp not in dict_emission[label]:
                dict_emission[label].append(timestamp)
    return dict_emission


def split_string(raw_string, list_delimiter):
    list_output = []
    for delimiter in list_delimiter:
        output = raw_string.split(delimiter)[0].strip()
        list_output.append(output)
        raw_string = raw_string.replace(output, "").replace(delimiter, "")
    list_output.append(raw_string.strip())
    return list_output

In [ ]:
## visualisation of results

# Create an Output widget to capture the IFrame
vis = widgets.Output(height='800px')

# Define a function to render the IFrame inside the Output widget
@vis.capture()
def render_iframe():
    display(IFrame(src=IFRAME_URL, width='100%', height='800px'))

# Call the function to render the IFrame
render_iframe()

In [ ]:
# Interactive frontend

# Create a text area widget
text_area = widgets.Textarea(
    placeholder="""Type name of a location to search for specific simulation.
    Leave blank and all available simulations will be returned.""",
    description="""Search for ship emission simulation data:""",
    layout=widgets.Layout(flex_flow='column',min_height='150px'),
    style={'description_width': 'initial'},
    disabled=False
)

# Create a button widget
submit_button = widgets.Button(
    description='Submit',
    button_style='success',
    tooltip='Submit',
    icon='check'
)

def print_with_indent(obj, indent=0):
    if isinstance(obj, dict):
        for key, value in obj.items():
            print('  ' * indent + str(key) + ':')
            print_with_indent(value, indent + 1)
    elif isinstance(obj, list):
        for index, item in enumerate(obj):
            print('  ' * indent + f"[{index}]:")
            print_with_indent(item, indent + 1)
    else:
        print('  ' * indent + str(obj))

# Define a function to handle the button click event
def on_button_clicked(b):

    with output_box:
        output_box.clear_output()
        # Print the text area content
        print_with_indent(parse_emission_data(get_emission_data(text_area.value)))


output_box = widgets.Output()

# Set the event handler for the button click event
submit_button.on_click(on_button_clicked)

# Arrange widgets in each column using VBox
column1 = widgets.VBox([text_area, submit_button, output_box],layout=widgets.Layout(width='50%',align_items='center'))
column2 = widgets.VBox([widgets.Label('Column 2')],layout=widgets.Layout(width='50%',align_items='center'))

# Create a horizontal box to hold the widgets
interactive = widgets.HBox([column1,column2],layout=widgets.Layout(width='100%'))

In [ ]:
# final-layout
table = widgets.VBox([vis,interactive],layout=widgets.Layout(align_items='center'),width='100%')

display(table)